In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
reviews_df = pd.read_csv('../data/books_rating.csv')
reviews_df.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [4]:
fiction_df = pd.read_csv('../data/books.csv')
fiction_df.head()

,title,synopsis,cover_link,year_published,info_link,average_rating,author,publisher,genre
0,I am the Clay,“[Chaim] Potok writes powerfully about the suf...,http://books.google.com/books/content?id=Tl6FU...,2010,https://play.google.com/store/books/details?id...,0,66,69,4
1,Bergdorf Blondes : A Novel,"Meet moi, 'a champagne bubble of a girl about ...",http://books.google.com/books/content?id=DmRMD...,2018,https://play.google.com/store/books/details?id...,0,342,33,1
2,O Little Town of Maggody: An Arly Hanks Mystery,"Country music, greed, and the unique madness o...",http://books.google.com/books/content?id=AUAED...,2016,https://play.google.com/store/books/details?id...,0,200,132,9
3,Little Myth Marker,"Skeeve, a young magician, and Aahz, his demon ...",http://books.google.com/books/content?id=Ta_z_...,2006,http://books.google.com/books?id=Ta_z_X30_eoC&...,0,360,5,3
4,Winter Winds (Seaside Seasons #4),"In Winter Winds, Gayle Roper concludes her del...",http://books.google.com/books/content?id=RQuZc...,2011,https://play.google.com/store/books/details?id...,0,149,119,10


In [5]:
reviews_df.rename(columns={'Title': 'title'}, inplace=True)
merged_df = pd.merge(fiction_df, reviews_df, on='title', how='left')
merged_df = merged_df[['title',  'profileName', 'review/score',  'review/text']]
merged_df.dropna(axis=0, inplace=True)
print(f'{merged_df.shape[0]} reviews')
merged_df.head(10)

20530 reviews


,title,profileName,review/score,review/text
0,I am the Clay,Adam Carr,4.0,. I enjoyed it thoroughly and would give it an...
1,I am the Clay,Adam Carr,4.0,I enjoyed it thoroughly and would give it an 8...
2,I am the Clay,Adam Carr,4.0,I enjoyed it thoroughly and would give it four...
5,I am the Clay,Delaney,5.0,"I read this book when I was 13, I couldn't qui..."
6,I am the Clay,SplitEdit,4.0,"This is a beautiful, thoughtful, moving book -..."
7,I am the Clay,A. Hale,1.0,I am an avid reader and liked/loved every book...
8,I am the Clay,"Nick ""Critic""",2.0,this is definitely not one of Chaim's greater ...
9,I am the Clay,"Prem Kumar ""Ashwin""",2.0,I Am The Clay is a story of suffering and civi...
10,I am the Clay,"Louis N. Gruber ""Author of Jay""",3.0,"The old man and the old woman, along with coun..."
11,I am the Clay,Billy Joe,2.0,Chaim Potok uses very vivid writing to portray...


In [6]:
reviewerCounts = merged_df.groupby(by='profileName')['profileName'].count().sort_values(ascending=False)
print(f'{reviewerCounts.shape[0]} reviewers')
reviewerCounts[reviewerCounts >= 20]


15375 reviewers


profileName
Harriet Klausner                    72
A Customer                          38
Blue Tyson "- Research Finished"    27
bernie "xyzzy"                      22
CoffeeGurl                          21
Name: profileName, dtype: int64

Pick a couple of obvious pseudnymous reviewers. Turns out there are a lot of duplicates, which suggests names are fake anyway. So remove duplicates of Title and profileName combinations.

In [35]:
reviews_df = merged_df[merged_df['profileName'].isin(['Harriet Klausner', 'A Customer', 'CoffeeGurl'])]
reviews_df.drop_duplicates(subset=['title', 'profileName'], inplace=True)
reviews_df.rename(columns={'review/score': 'rating', 'review/text': 'review'}, inplace=True)
reviews_df['rating'] = reviews_df['rating'].astype(int)
print(f'{reviews_df.shape[0]} reviews')


102 reviews


Make IDs for users and books

In [36]:
import json
with open('../data/users.json', 'r')as FILE:
    users = json.load(FILE)
user_list = [user['username'] for user in users]
reviews_df['user_id'] = reviews_df['profileName'].apply(lambda x: user_list.index(x)+1)

title_list = fiction_df['title'].tolist()
reviews_df['book'] = reviews_df['title'].apply(lambda x: title_list.index(x)+1)

reviews_df.head()

,title,profileName,rating,review,user_id,book
473,The Jury (Paul Madriani Novels),Harriet Klausner,5,"While floating in the water near San Diego, re...",1,6
1038,Key of Knowledge (Key Trilogy),Harriet Klausner,5,"In Pleasant valley, Pennsylvania, three women ...",1,30
1042,Key of Knowledge (Key Trilogy),A Customer,1,This was my least favorite book in a poorly wr...,2,30
1312,Lover's Talisman,CoffeeGurl,4,This book broke the reading rut I've had for a...,3,35
1347,The Atrocity Archives (Laundry Files),Harriet Klausner,5,"""The Atrocity Archive"". .The mission of the ul...",1,36


Create CSV file of Reviews

In [37]:
reviews_df.drop(columns=['profileName', 'title'], inplace=True)
reviews_df.to_csv('../data/Reviews.csv', index=False)

Build sample reader logs

In [38]:
reader_logs = []
for _, row in reviews_df.iterrows():
    log = {
        'user_id': row['user_id'],
        'book': row['book'],
        'status': 'COVER2COVER'
    }
    reader_logs.append(log)

reader_logs_df = pd.DataFrame(reader_logs)
reader_logs_df.to_csv('../data/ReaderLogs.csv', index=False)
reader_logs_df.head()


,user_id,book,status
0,1,6,COVER2COVER
1,1,30,COVER2COVER
2,2,30,COVER2COVER
3,3,35,COVER2COVER
4,1,36,COVER2COVER
